# Mask R-CNN Training Demo

This notebook demonstrates the steps for training Mask R-CNN models. 
Mask R-CNN is a convolution-based neural network for the task of object instance segmentation. The original paper describing the model can be found [here](https://arxiv.org/abs/1703.06870). NVIDIA’s Mask R-CNN is an optimized version of [Facebook’s implementation](https://github.com/facebookresearch/maskrcnn-benchmark), leveraging mixed precision arithmetic and tensor cores for faster training times while maintaining comparable accuracy with single precision, i.e. FP32, training.

## Notebook  Content
1. [Pre-requisite: data](#1)
1. [Pre-requisite: container](#2)
1. [Training](#3)
1. [Testing trained model](#4)


<a id="2"></a>
## 1. Pre-requisite: data

This notebook demonstrates training and validation or the Mask R-CNN model on the [COCO 2014 dataset](http://cocodataset.org/#download). If not already available locally, the following [script](../../download_dataset.sh) in this repository provides a convinient way to download and extract all the necessary data in one go:

```
./download_dataset.sh /path/to/coco-2014
```

This script can be ran on a Linux host machine. Be mindful of the size of the raw data (~20GB). The script makes use of `wget` and will automatically resume if disrupted. Once downloaded, the scrip invoke `dtrx` to extract the data. The final data directory should look like:

```
/path/to/coco-2014
  annotations/
    instances_train2014.json
    instances_val2014.json
  train2014/
    COCO_train2014_*.jpg
  val2014/
    COCO_val2014_*.jpg
```

<a id="2"></a>
## 2. Pre-requisite: container

The most convinient way to make use of the NVIDIA Mask R-CNN model is via a docker container, which provides a self-contained, isolated and re-producible environment for all experiments. First, clone the repository:

```
git clone https://github.com/NVIDIA/DeepLearningExamples.git
cd DeepLearningExamples/PyTorch/Segmentation/MaskRCNN
```

Next, build the NVIDIA Mask R-CNN container:

```
cd pytorch/docker/docker-jupyter
docker build --rm -t nvidia_maskrcnn_jupyter .
```

Then launch the container with:

```
PATH_TO_COCO='/path/to/coco-2014'
MOUNT_LOCATION='/datasets/data'
NAME='maskrcnn_jupyter'

docker run --runtime=nvidia -p 8888:8888 -v $PATH_TO_COCO:/$MOUNT_LOCATION --rm --name=$NAME --shm-size=1g --ulimit memlock=-1 --ulimit stack=67108864 --ipc=host nvidia_maskrcnn_jupyter
```

Open the Jupyter GUI interfact on your host machine at http://localhost:8888. The default password is empty.

We verify that the data has been properly downloaded and mounted.

In [3]:
!ls /datasets/data -l

total 20012368
drwxr-xr-x 2 218419156 1876110778        4096 Jun  3 04:53 annotations
drwxr-xr-x 2 root      root              4096 Jun  3 04:53 annotations.1
-rw-rw-r-- 1      1002       1002   252872794 Jul 10  2018 annotations_trainval2014.zip
-rw-rw-r-- 1      1002       1002    77061279 Dec 21 05:20 coco_annotations_minival.tgz
drwxrwxr-x 2 root      root           4763648 Aug 15  2014 train2014
-rw-rw-r-- 1      1002       1002 13510573713 Jul 11  2018 train2014.zip
drwxrwxr-x 2 root      root           2351104 Aug 15  2014 val2014
-rw-rw-r-- 1      1002       1002  6645013297 Jul 11  2018 val2014.zip


<a id="3"></a>
## 3. Training

The bash script [bash scripts](../scripts/train.sh) provides a convinient interface to 